In [1]:
import requests
import pandas as pd
import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
api_key = 'Sxwiysbra37brnJK9Fp12Z81vXvU0IEN'

In [3]:
def df_search_item(itemName = '', item_hashtag = ''):
    url = 'https://api.neople.co.kr/df/items'

    params = {
        'limit' : 30,
        'apikey' : api_key,
        'wordType' : 'match'
    }

    if itemName != '':
        params['itemName'] = itemName
    if item_hashtag != '':
        params['hashtag'] = item_hashtag

    return requests.get(url, params=params)

def df_aution_item(itemName, limit=100):
    url = f'https://api.neople.co.kr/df/auction'
    params = {
        'itemName' : itemName,
        'limit' : limit,
        'sort' : 'auctionNo:desc',
        'apikey' : api_key
        
    }
    
    return requests.get(url, params=params)

def df_aution_sold(itemName, limit=100):
    url = f'https://api.neople.co.kr/df/auction-sold'
    params = {
        'itemName' : itemName,
        'limit' : limit,
        'apikey' : api_key
    }
    
    return requests.get(url, params = params)

In [33]:
def check_new_items(check_cols, new_check_lst, isFirst, check_type, itemId = ''):

    
    new_items = []
    
    if check_type == 'regi':

        if isFirst:     # 계속 켜놓지 못하니까 불러와서 확인하기 
            print('Frist Search in REGI')
            auctionIds = get_data_from_db(check_type)

            for check_value in check_cols:
                if str(check_value) not in auctionIds:
                    new_items.append(check_value)
                new_check_lst.append(check_value)   

        
        else:
            for check_value in check_cols:
                if check_value not in new_check_lst:
                    new_items.append(check_value)
                    new_check_lst.append(check_value)
                
    elif check_type == 'saled':
        if isFirst:
            print('Frist Search in SALED')
            db_saled_dates = get_data_from_db(check_type, target = itemId)
            
            for check_value in check_cols:
                if check_value not in db_saled_dates:
                    new_items.append(check_value)
                new_check_lst[itemId].append(check_value)   

        else:
            for check_value in check_cols:
                if check_value not in new_check_lst[itemId] :
                    new_items.append(check_value)
                    new_check_lst[itemId].append(check_value)
                
    return new_items

def regi_item(df, check_type):
    
    if check_type == 'regi':
        use_cols = ['auctionNo', 'regDate', 'itemId', 'count', 'unitPrice']
        
    elif check_type == 'saled':
        use_cols = ['soldDate', 'itemId', 'count', 'unitPrice']
        
    excecuteDB(df[use_cols], check_type)
    

# 경매장에서 아이템 검색
def searching_item(target, check_type):
    
    if check_type == 'regi':
        return pd.DataFrame(df_aution_item(target).json()['rows']) 
    
    elif check_type == 'saled':
        return pd.DataFrame(df_aution_sold(target).json()['rows']) 


def search_and_regi_item(target, new_check_regi_lst, new_check_saled_lst, isFrist, check_type):
    
    df = searching_item(target, check_type)
    
    if len(df) == 0:
#         print('No Data')
        return 
    
    # 경매장 등록 새로운 아이템 체크
    if check_type == 'regi':
        new_item = df['auctionNo']
        new_item = check_new_items(new_item, new_check_regi_lst, isFrist, check_type)
        df = df[df['auctionNo'].isin(new_item)] # 새롭게 등록된 아이템 필터링

    # 경매장 판매 새로운 아이템 체크
    elif check_type == 'saled':
        new_item = df['soldDate']
        new_item = check_new_items(new_item, new_check_saled_lst, isFrist, check_type, itemId = item_dicts[target])
        df = df[df['soldDate'].isin(new_item)] # 새롭게 등록된 아이템 필터링
        
    if len(df) > 0:
        print(f'{check_type} {target} New {len(new_item)} items')  
    regi_item(df, check_type) # DB등록

In [18]:
import  mysql.connector

def init_mysql():
    aws = mysql.connector.Connect(
        database = 'df',
        host = "dungeonandfighter-db.crnv1am9ifjd.ap-northeast-2.rds.amazonaws.com",
        port = 3306,
        user = 'sold_table',
        password = '1234'
    )
    return aws

In [8]:
# 등록되어있는 기록의 옥션ID정보 가져오기(리스트)
def get_data_from_db(check_type, target = ''):
    cur = aws.cursor()
    
    if check_type == 'regi':
        query = 'SELECT auctionNo FROM ticket_history'
        cur.execute(query)
        return pd.DataFrame(cur.fetchall(), columns={'auctionNo'})['auctionNo'].values
    
    elif check_type == 'saled':
        query = f"Select soldDate from sold_history where itemId like '{target}'"
        cur.execute(query)
        return pd.DataFrame(cur.fetchall(), columns={'soldDate'})['soldDate'].values

In [9]:
def excecuteDB(df, check_type):
    cur = aws.cursor()
    cols = df.columns
    insert_cols = ",".join(cols)
    
    for idx, row in df.iterrows():
        insert_value = ['"'+str(row[col])+'"' for col in cols]
        insert_value = ",".join(map(str,insert_value))
        
        query = ''
        
        if check_type == 'regi':
            query = 'INSERT INTO ticket_history('+ insert_cols +') VALUES('+ insert_value +');'
            
        elif check_type == 'saled':
            query = 'INSERT INTO sold_history('+ insert_cols +') VALUES('+ insert_value +');'
        
        if query != '':
            cur.execute(query)
        
    aws.commit()
    

In [10]:
# 공홈 오늘의 던파 직업별 플래티넘 엠블렘 정리 참고.


skill_keywords = [
    '중화기 마스터리', '오버부스트 팩', 'APG-63', # 런처 - 디스트로이어, 스톰트루퍼
    '스타일리쉬', '리볼버 강화', # 레인저 - 레이븐, 크림슨 로제
    '아크리액터', '로봇 엔지니어링', '트랜스포메이션', '하이테크놀로지', # 메카닉 - 프라임, 옵티머스
    '병기 숙련', '매거진 드럼', '유탄 마스터리', # 스핏파이어 - 커맨더, 프레이야
    '강렬한 테이크 다운', '임팩트 콤비네이션', '파워 슬램', # 그래플러 - 그래플러, 얼티밋 디바
    '나선의 넨', '뇌명 : 사나운 빛의 넨수', # 넨마스터 - 넨마스터, 염제 폐월수화
    '머슬 시프트', '급소 지정', # 스트라이커 - 스트라이커, 카이저
    '도발', '클로 마스터리', # 스트리트파이터 - 스트리트파이터, 용독문주
    '청혈투귀', '검귀의 도 마스터리', # 검귀
    '혈십자', '프렌지', '갈증', # 버서커
    '달의 커튼', '몽롱한 눈의 브레멘', # 소울브링어
    '정신이 번쩍!', '파동각인', #아수라
    '검기상인', '무기의 극의', # 웨펀마스터
    '마검 제어', '파천세', # 데몬슬레이어
    '삼화취정', # 베가본드
    '속성의 소검 마스터리', '쾌속의 도 마스터리', '파쇄의 둔기 마스터리', '견고의 대검 마스터리', '마검일체', # 소드마스터
    '우시르의 저주', '이보브', # 다크템플러
    '대식가', '아스트라 소환', # 드레드노트
    '체인러쉬', '강인한 신념', #엘븐나이트
    '혼혈', '힘의 논리', # 마신
    '세라픽 페더', '세라픽 폴', # 세이비어
    '단검 마스터리', '쌍검 마스터리', # 알키오네
    '발라크르의 맹약', '어둠의 고리', # 타나토스
    '날카로운 단검', '암살기술', #그림리퍼
    '인법 : 잔영 남기기', '화염의 인', '차크라 웨펀 숙련', #시라누이
    '잠식', '어둠의 투창 마스터리', '마창 폭주', # 에레보스
    '장창 숙련', '미라지 부스트', # 듀란달
    '광창 마스터리', '헌터즈 샤프니스', # 제노사이더
    '임팩트 스매쉬', '마창 제어', #워로드
    '괴리 : 디멘션 할로우', '도그마 디바이스', #어센션
    '블러드', '블러드 혼', # 뱀파이어 로드
    '발현', '프로즌웨폰 마스터리', '아이스 로드', # 이터널
    '휘몰아치는 질풍의 봉 마스터리', '태풍의 전조', # 아이올로스
    '원소의 이해', '원소 융합', '엘레멘탈 실드', # 오블리비언
    '퍼밀리어와 친하게 지내기', '스위트 캔디바', # 지니위즈
    '창 마스터리', '봉 마스터리', '니우의 전투술', # 아슈타르테
    '교감', '환수 강화 오라', # 이클립스
    '속성 마스터리', #오버마인드
    '퍼페티어', '사악한 호기심', # 헤카테
    '코어 블레이드 마스터리', '인젝션', #패스파인더
    '소태도 마스터리', '요원의 전투술', #레퀴엠
    '중검 마스터리', '화약 다루기', #언터처블
    '장도 마스터리', '숙련된 전문가', #갓파더
    '환청', '낫 마스터리', #이모탈
    '윌 드라이버', '테크니컬 마스터리', # 저스티스
    '수호의 은총', '보호의 징표', '용기의 은총', '굳건한 의지', # 세인트
    '거병 마스터리', '퇴마의 서', # 태을선인
    '법력 강화', '천녀기우제', #천선낭랑
    '죄업에 짓눌린 육신', '짊어진 자의 낫 마스터리', #리디머
    '성화', '배틀액스 마스터리', #인페르노
    '계시 : 아리아', '십자가 마스터리', '수호의 축복', # 세라핌
    '어둠의 도 마스터리', '어둠의 둔기 마스터리', '어둠의 소검 마스터리', '어둠의 대검 마스터리', '어둠의 광검 마스터리', # 다크나이트
    '계열강화', '상상력' # 크리에이터
]

In [11]:
job_platinum_emblem_names = []
search_keyword = '플래티넘 엠블렘['

for skill in skill_keywords:
    job_platinum_emblem_names.append(search_keyword + skill+']')


In [28]:
job_platinum_emblem_ids = []
item_dicts = dict()

for job_p_e in job_platinum_emblem_names:
    tmp = pd.DataFrame(df_search_item(itemName=job_p_e).json()['rows'])
    for idx, itemid in enumerate(tmp['itemId'].values):
        if idx != 0:
            item_dicts[job_p_e + str(idx)] = itemid
        else:
            item_dicts[job_p_e] = itemid
        job_platinum_emblem_ids.append(itemid)

# item_dicts = {item_name : item_id for item_name, item_id in zip(job_platinum_emblem_names, job_platinum_emblem_ids)}

item_dicts

{'플래티넘 엠블렘[중화기 마스터리]': 'a168829a90efd41a0ec9f767ce7de0d2',
 '플래티넘 엠블렘[중화기 마스터리]1': 'c8a1b32c06de4756837a1c723cb89577',
 '플래티넘 엠블렘[오버부스트 팩]': 'bc7bebcbaccd54572255d3454587b2f5',
 '플래티넘 엠블렘[APG-63]': '45226ebb2a84f8a64a6372a38f55cd78',
 '플래티넘 엠블렘[스타일리쉬]': '224cebf19acfc1430287e817d88c8edb',
 '플래티넘 엠블렘[스타일리쉬]1': 'a5a06612a631777aa8a16e98b63a06d7',
 '플래티넘 엠블렘[리볼버 강화]': '7a6e818b45a653403769cbeb9c9019f4',
 '플래티넘 엠블렘[리볼버 강화]1': 'bb46258e9e00d8c6aab5dc3358d53f32',
 '플래티넘 엠블렘[리볼버 강화]2': '71a6f1d147ac21f002933157ce5f76fd',
 '플래티넘 엠블렘[리볼버 강화]3': '9329bebd64b9975b15fbe730788dcc14',
 '플래티넘 엠블렘[리볼버 강화]4': 'e1145188a4c161a56955d826e7a2b1d9',
 '플래티넘 엠블렘[아크리액터]': 'b3ed45afc1162e0ae1a866f3218832d8',
 '플래티넘 엠블렘[로봇 엔지니어링]': '0358bc3a448b3e3760b32e4979676eb8',
 '플래티넘 엠블렘[트랜스포메이션]': '9077665b40cb54148ad96130d87faa99',
 '플래티넘 엠블렘[하이테크놀로지]': '9ed80be80f9d09561200f29110d026a9',
 '플래티넘 엠블렘[병기 숙련]': 'e4395fa332a5ac427722e38700b6e327',
 '플래티넘 엠블렘[병기 숙련]1': '26fe1c8fb3d78eb1ece2441970cd75d4',
 '플래티넘 엠블렘[병기 숙련]2':

In [34]:
new_check_regi_lst = []
new_check_saled_lst = {search_id : [] for search_id in item_dicts.values()}
search_type = ['regi', 'saled']
isFrist = True

aws = init_mysql()

while True:
    for target in item_dicts.keys():
#         try:
        for check_type in search_type:
            search_and_regi_item(target, new_check_regi_lst, new_check_saled_lst, isFrist, check_type = check_type)
#         except Exception as e:
#             print(f"{target} Error -> ", e)
#             time.sleep(1)
            
    isFrist = False
    time.sleep(0.3)

Frist Search in REGI
Frist Search in SALED
saled 플래티넘 엠블렘[중화기 마스터리] New 44 items
Frist Search in REGI
Frist Search in SALED
Frist Search in SALED
saled 플래티넘 엠블렘[APG-63] New 1 items
Frist Search in REGI
Frist Search in SALED
saled 플래티넘 엠블렘[스타일리쉬] New 37 items
Frist Search in REGI
Frist Search in SALED
saled 플래티넘 엠블렘[리볼버 강화] New 100 items
Frist Search in REGI
Frist Search in SALED
Frist Search in REGI
Frist Search in SALED
Frist Search in REGI
Frist Search in SALED
Frist Search in REGI
Frist Search in SALED
saled 플래티넘 엠블렘[하이테크놀로지] New 1 items
Frist Search in REGI
Frist Search in SALED
saled 플래티넘 엠블렘[병기 숙련] New 52 items
Frist Search in SALED
Frist Search in REGI
Frist Search in SALED
saled 플래티넘 엠블렘[유탄 마스터리] New 22 items
Frist Search in REGI
Frist Search in SALED
saled 플래티넘 엠블렘[강렬한 테이크 다운] New 54 items
Frist Search in SALED
Frist Search in SALED
Frist Search in REGI
regi 플래티넘 엠블렘[나선의 넨] New 1 items
Frist Search in SALED
saled 플래티넘 엠블렘[나선의 넨] New 57 items
Frist Search in REGI
Frist Search i

saled 플래티넘 엠블렘[환수 강화 오라] New 2 items
regi 플래티넘 엠블렘[계시 : 아리아] New 1 items
regi 플래티넘 엠블렘[계열강화] New 1 items
regi 플래티넘 엠블렘[트랜스포메이션] New 1 items
saled 플래티넘 엠블렘[임팩트 스매쉬] New 1 items
regi 플래티넘 엠블렘[오버부스트 팩] New 1 items
regi 플래티넘 엠블렘[병기 숙련] New 1 items
regi 플래티넘 엠블렘[퍼페티어] New 1 items
regi 플래티넘 엠블렘[퍼밀리어와 친하게 지내기] New 1 items
regi 플래티넘 엠블렘[봉 마스터리] New 1 items
regi 플래티넘 엠블렘[수호의 은총] New 1 items
regi 플래티넘 엠블렘[아스트라 소환] New 1 items
saled 플래티넘 엠블렘[스위트 캔디바] New 2 items
saled 플래티넘 엠블렘[쾌속의 도 마스터리] New 1 items
saled 플래티넘 엠블렘[수호의 축복] New 2 items
saled 플래티넘 엠블렘[속성 마스터리] New 1 items
regi 플래티넘 엠블렘[힘의 논리] New 1 items
regi 플래티넘 엠블렘[교감] New 1 items
regi 플래티넘 엠블렘[속성의 소검 마스터리] New 1 items
regi 플래티넘 엠블렘[마검일체] New 1 items
regi 플래티넘 엠블렘[프렌지] New 1 items
regi 플래티넘 엠블렘[헌터즈 샤프니스] New 1 items
saled 플래티넘 엠블렘[계시 : 아리아] New 1 items
saled 플래티넘 엠블렘[속성 마스터리] New 1 items
regi 플래티넘 엠블렘[세라픽 페더] New 1 items
regi 플래티넘 엠블렘[뇌명 : 사나운 빛의 넨수] New 1 items
regi 플래티넘 엠블렘[대식가] New 1 items
regi 플래티넘 엠블렘[달의 커튼] New 1 items
regi 플래티넘 엠블렘[정신이 번쩍

regi 플래티넘 엠블렘[계시 : 아리아] New 1 items
regi 플래티넘 엠블렘[수호의 축복] New 1 items
regi 플래티넘 엠블렘[중화기 마스터리] New 1 items
regi 플래티넘 엠블렘[임팩트 콤비네이션] New 1 items
regi 플래티넘 엠블렘[뇌명 : 사나운 빛의 넨수] New 1 items
regi 플래티넘 엠블렘[검귀의 도 마스터리] New 1 items
regi 플래티넘 엠블렘[견고의 대검 마스터리] New 1 items
regi 플래티넘 엠블렘[뇌명 : 사나운 빛의 넨수] New 1 items
saled 플래티넘 엠블렘[스타일리쉬] New 2 items
regi 플래티넘 엠블렘[유탄 마스터리] New 1 items
saled 플래티넘 엠블렘[몽롱한 눈의 브레멘] New 2 items
regi 플래티넘 엠블렘[환수 강화 오라] New 1 items
regi 플래티넘 엠블렘[계시 : 아리아] New 1 items
regi 플래티넘 엠블렘[뇌명 : 사나운 빛의 넨수] New 1 items
regi 플래티넘 엠블렘[마검 제어] New 1 items
saled 플래티넘 엠블렘[단검 마스터리] New 1 items
saled 플래티넘 엠블렘[마창 제어] New 1 items
regi 플래티넘 엠블렘[굳건한 의지] New 1 items


KeyboardInterrupt: 

In [13]:
aws.close()